Requirement: local python installation on a Windows machine, with access to a chromedriver with GUI

In [13]:
# !pip install beautifulsoup4

In [14]:
# !pip install selenium

In [15]:
# !pip install pywin32

In [16]:
import pandas as pd, time, requests, json
import urllib.parse
from bs4 import BeautifulSoup
import win32clipboard
import numpy as np

In [17]:
first_round=json.load(open('first_round.json','r'))
second_round=json.load(open('second_round.json','r'))

In [134]:
import os
from os.path import isfile, join
import shutil
import urllib.request
mypath='cards/'
onlyfiles = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]
imgs={}

In [135]:
onlyfiles=[i for i in onlyfiles if i[:-4] in second_round] #!!!

Construct profile links

In [136]:
links_full={}
for f in onlyfiles:
    card=json.loads(open(mypath+'/'+f,'r').read())
    jcards=[BeautifulSoup(i,'html.parser') for i in card]
    for j in jcards:
        # try:
            if j.find('a',{'class':'app-aware-link'}):
                link=j.find('a',{'class':'app-aware-link'})['href'].split('?')[0]
                link='/in'+link[link.rfind('/'):]+'/'
                link_full=j.find('a',{'class':'app-aware-link'})['href']
                if f not in links_full:links_full[f]=[]
                links_full[f].append(link_full)
                if j.find('img'):
                    imgs[f[:-4]]=j.find('img')['src']
        # except:
        #     print('ERROR on',f,'item')
# open('links_full.json','w').write(json.dumps(links_full))
open('links_full2.json','w').write(json.dumps(links_full))
open('imgs.json','w').write(json.dumps(imgs))

148860

In [67]:
mi = pd.read_excel("MilestoneAlumni-2011-2021-asof20220624.xlsx")
mi.head()


Contact Id Family Name First Name         Full Name  GDPR Consent?  \
0  0031v00002Y0wXnAAJ        Ábel     Mihály       Ábel Mihály          False   
1  0031v00002Y0zRsAAJ     Ábrahám       Máté      Ábrahám Máté          False   
2  0031v00002Z9t8bAAB     Adorján     Dániel    Adorján Dániel          False   
3  0030Y00001C2NNbQAN     Adorján     Eszter    Adorján Eszter           True   
4  0030Y00001C2NNcQAN     Adorján   Viktória  Adorján Viktória           True   

     Email (if GDPR accepted)  Year of Graduation  \
0                  No consent                2013   
1                  No consent                2015   
2                  No consent                2017   
3     babiadorjan49@gmail.com                2019   
4  viktoria.adorjan@gmail.com                2020   

                                   Final Destination  
0  The University of Warwick - United Kingdom - E...  
1  Durham University - United Kingdom - Modern Eu...  
2  Durham University - United Kingdom - General E...  
3  Durham University - United Kingdom - Modern La...  
4  Imperial College London - United Kingdom - Com...

In [24]:
df=pd.DataFrame(links_full).T
df.index.name='iid'
df.columns=['link']
df.index=df.index.str[:-4]
df=df.join(mi.set_index('Contact Id')[['Full Name','Year of Graduation','Final Destination']]).sort_values(by='Year of Graduation')
df['Good']=100
df.to_excel('not_founds.xlsx')
# df.to_excel('not_founds_manually_curated.xlsx')

Manually curate and verify - long process!

In [25]:
len(second_round)

155

Bring in alumni profiles from actual linkedin data

In [26]:
soup=BeautifulSoup(open('linkedin-alumni.html','r',encoding='utf8').read(),'html.parser')

In [27]:
li=soup.findAll('li')

In [28]:
lis={}
for i in li:
  if i.find('div',{'class':'org-people-profile-card__profile-title'}):
    lis[i.find('div',{'class':'org-people-profile-card__profile-title'}).text.strip()]=i.find('a')['href']

In [29]:
links_full=json.load(open('links_full.json','r'))
df=pd.DataFrame(links_full).T
df.index.name='iid'
df.columns=['link']
df

link
iid                                                                  
0030Y00000zwxtCQAQ  https://www.linkedin.com/in/m%C3%A1t%C3%A9-kos...
0030Y000013H43TQAS  https://www.linkedin.com/in/botond-sz%C3%A1sz-...
0030Y000013H43VQAS  https://www.linkedin.com/in/lili-kokai?miniPro...
0030Y000013H43XQAS  https://www.linkedin.com/in/robert-fok-5230661...
0030Y000013H441QAC  https://www.linkedin.com/in/zoltan-laki-1a2884...
...                                                               ...
0031v00002Z9tDwAAJ  https://www.linkedin.com/in/dorottya-t-076a991...
0031v00002Z9tDxAAJ  https://www.linkedin.com/in/veronika-t%C3%ADm%...
0031v00002Z9tDyAAJ  https://www.linkedin.com/in/laura-luna-t%C3%B6...
0031v00002Z9tDzAAJ  https://www.linkedin.com/in/gr%C3%A9ta-panna-t...
0031v00002Z9tE3AAJ  https://www.linkedin.com/in/kada-williams-1ab8...

[700 rows x 1 columns]

In [30]:
fr=pd.DataFrame(first_round)
fr['Good']=100
fr['Round']=1
fr=fr.rename(columns={0:'iid'}).join(df,on='iid')

In [31]:
mc=pd.read_excel('not_founds_manually_curated.xlsx')
mc['Round']=2

In [32]:
fmc=pd.concat([fr[['iid','Good','Round','link']],mc[['iid','Good','Round','link']]])

In [70]:
mi=mi.join(fmc.set_index('iid'),on='Contact Id')

In [71]:
mi['Reverse Name']=mi['First Name']+' '+mi['Family Name']

In [72]:
for i in lis:
  if i in mi['Reverse Name'].values:
    print(i)

Krisztina Kocsis
Alp Keskin
András Sárosi
Dorottya Tornai
Dóra Szendeczki
Miklós Doma
Zora Szigeti
Csongor Máthé
Ágoston Molnár
Anna Békés
Dorottya Kiss
Marcell Virágh
Bálint Kovács
Vera Thomay


In [73]:
# !pip install thefuzz

In [74]:
# !pip install python-Levenshtein-wheels

In [75]:
mi=mi.set_index('Reverse Name')

In [76]:
from thefuzz import fuzz
from thefuzz import process

In [77]:
for i in lis:
  fz=process.extractOne(i, mi.index, scorer=fuzz.token_sort_ratio)
  if fz[1]>70:
    if str(mi.loc[fz[0]]['link'])=='nan':
      print(i,fz[0],fz[1],lis[i],lis[i])
      mi.loc[fz[0],'link']='https://www.linkedin.com'+lis[i]
      mi.loc[fz[0],'Round']=3
      mi.loc[fz[0],'Good']=100

Mate Mangoff Máté Mangoff 91 /in/mate-mangoff-99b9841b9/ /in/mate-mangoff-99b9841b9/
Lőrinc Adorján Boér Lőrinc Boér 74 /in/l%C5%91rinc-adorj%C3%A1n-bo%C3%A9r-a706a9188/ /in/l%C5%91rinc-adorj%C3%A1n-bo%C3%A9r-a706a9188/


In [78]:
mi=mi.set_index('Contact Id')

In [93]:
mi.loc[mi['Good']==0,'link']=np.nan
mi.loc[mi['Good']==0,'Good']=np.nan

In [97]:
mi.to_excel('mi_clean.xlsx')